In [1]:
import bw2data, bw2calc, bw2io
from edges import EdgeLCIA, get_available_methods
from bw2analyzer.contribution import ContributionAnalysis

In [2]:
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

In [3]:
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
#act = [a for a in bw2data.Database("ecoinvent-3.10-cutoff") if "NMC" in #a["name"]][0]
act

'evaporation of natural gas, import from ID' (cubic meter, KR, None)

In [4]:
#method = ('GeoPolRisk', '2024')
method = ('AWARE 1.2c', 'Country', 'non', 'irri', 'yearly')
LCA = EdgeLCIA({act: 1}, method)
LCA.lci()
LCA.lcia()
LCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


+-------------------------------------+----------------------------------------------------+
|             Method name             | ('AWARE 1.2c', 'Country', 'non', 'irri', 'yearly') |
|              Data file              |         AWARE 1.2c_Country_non_irri_yearly         |
|       Number of CFs in method       |                        3420                        |
|          Number of CFs used         |                        572                         |
|  Number of exchanges characterized  |                        8877                        |
| Number of exchanges uncharacterized |                         0                          |
+-------------------------------------+----------------------------------------------------+


0.025975336750489575

In [5]:
LCA.generate_cf_table()

Matrix type: biosphere


,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Water, cooling, unspecified natural origin","(natural resource, in water)","particleboard production, uncoated, from virgi...","particleboard, uncoated",RoW,1.044026e-10,20.179399,2.106782e-09
1,"Water, cooling, unspecified natural origin","(natural resource, in water)","heat and power co-generation, lignite","electricity, high voltage",SI,8.116760e-07,1.087201,8.824547e-07
2,"Water, cooling, unspecified natural origin","(natural resource, in water)",atrazine production,atrazine,RoW,2.020677e-11,19.819034,4.004786e-10
3,"Water, cooling, unspecified natural origin","(natural resource, in water)","sheet rolling, copper","sheet rolling, copper",RoW,2.248532e-09,19.819034,4.456374e-08
4,"Water, cooling, unspecified natural origin","(natural resource, in water)","phosphoric acid production, dihydrate process","phosphoric acid, fertiliser grade, without wat...",MA,2.137708e-07,54.031045,1.155026e-05
...,...,...,...,...,...,...,...,...
8872,Water,"(water, surface water)","kerosene production, petroleum refinery operation",kerosene,BR,4.541320e-10,-1.898993,-8.623936e-10
8873,Water,"(water, surface water)",treatment of residue from mechanical treatment...,"residue from mechanical treatment, IT accessory",CH,5.577962e-20,-0.737323,-4.112761e-20
8874,Water,"(water, surface water)","treatment of sewage sludge, 70% water, WWT-SLF...","sewage sludge, 70% water, WWT-SLF, waste polye...",GLO,1.118355e-15,-19.820964,-2.216687e-14
8875,Water,"(water, surface water)","petroleum and gas production, onshore","natural gas, high pressure",VE,1.644462e-09,-4.584610,-7.539218e-09


In [7]:
LCA.ignored_locations

set()

In [9]:
method = ('AWARE 1.2c', 'Country', 'mixed', 'yearly')
convmethod = (
  # 'ecoinvent-3.10',
  'EF v3.1 EN15804',
  'water use',
  'user deprivation potential (deprivation-weighted water consumption)')

In [10]:
import bw2calc

convLCA = bw2calc.LCA({act: 1}, convmethod)
convLCA.lci()
convLCA.lcia()
convLCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


0.06176615560312869